In [1]:
import pandas as pd
from unidecode import unidecode
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, lit, lower

In [2]:
spark = SparkSession.builder.appName('MyApp').getOrCreate()

23/03/18 14:00:57 WARN Utils: Your hostname, MacBook-Air-di-Emanuele.local resolves to a loopback address: 127.0.0.1; using 10.0.0.197 instead (on interface en0)
23/03/18 14:00:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/18 14:00:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/18 14:00:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#wiki = pd.read_csv("../datasets/wiki_movie_plots_deduped.csv")

wiki = spark.read.option("escape", "\"").csv('../datasets/wiki_movie_plots_deduped.csv',header=True, inferSchema=True)
wiki

AnalysisException: Path does not exist: file:/Users/manu/PycharmProjects/Big-Data-Project/datasets/wiki_movie_plots_deduped.csv

In [3]:
#train = pd.read_csv("../datasets/gpt_data/train_plot_awards.csv", sep=',')
train = spark.read.option("escape", "\"").csv("../../datasets/gpt_data/train_plot_awards.csv", sep=",", header=True, inferSchema=True)
train

,_c0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label,plot,awards
0,4,tt0010600,The Doll,Die Puppe,1919,\N,66,1898.0,True,The Doll is a 1919 silent film directed by E...,0
1,7,tt0011841,Way Down East,Way Down East,1920,\N,145,5376.0,True,The Way Down East is a 1920 silent film dire...,1
2,9,tt0012494,Déstiny,Der müde Tod,1921,\N,97,5842.0,True,Destiny is a 1921 German silent fantasy film...,0
3,25,tt0015163,The Navigator,The Navigator,1924,\N,59,9652.0,True,The Navigator is a 1924 silent comedy film s...,0
4,38,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925,\N,93,17887.0,True,The Phantom of the Opera is a 1925 silent ho...,0
...,...,...,...,...,...,...,...,...,...,...,...
7954,9966,tt9625664,Trauma Center,NaN,2019,\N,87,12951.0,False,The Trauma Center is a 2019 drama film direc...,0
7955,9981,tt9741310,Slaxx,Slaxx,2020,\N,77,2464.0,False,Slaxx is a 2020 horror-comedy about a posses...,0
7956,9982,tt9742392,Kindred,Kindred,2020,\N,101,1719.0,False,The Kindred is a 2020 science fiction horror...,0
7957,9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,NaN,2020,\N,111,4144.0,True,The Bee Gees: How Can You Mend a Broken Hear...,0


In [4]:
#train["startYear"].replace(r"\N", None, inplace=True)
#train["startYear"].fillna(train['endYear'], inplace=True)
#train["startYear"] = train["startYear"].astype(int)

train = train.withColumn("startYear", regexp_replace(col("startYear"), r"\\N", str(None)))
train = train.fillna({'startYear': 'endYear'}, subset=['startYear'])
#train = train.select("startYear").fillna(train.select("endYear"))
train = train.withColumn("startYear", col("startYear").cast("int"))

In [5]:
#train["primaryTitle"] = train["primaryTitle"].str.lower()
#train['primaryTitle'] = train['primaryTitle'].apply(unidecode)

train = train.withColumn("primaryTitle", lower(col("primaryTitle")))
train = train.withColumn("primaryTitle", unidecode(train["primaryTitle"]))

In [3]:
#wiki["Title"] = wiki["Title"].str.lower()
#wiki.rename({"Title": "primaryTitle"}, axis=1, inplace=True)
#wiki.rename({"Release Year": "startYear"}, axis=1, inplace=True)
#wiki['primaryTitle'] = wiki['primaryTitle'].apply(unidecode)

wiki = wiki.withColumn("title", lower(col("Title")))
wiki.withColumnRenamed("primaryTitle", "Title")
wiki.withColumnRenamed("startYear", "Release Year")
wiki = wiki.withColumn("primaryTitle", unidecode(wiki["primaryTitle"]))

In [ ]:
# remove rows where the 'name' column is a duplicate
#wiki = wiki.drop_duplicates(subset=['primaryTitle'], keep='last')

wiki = wiki.dropDuplicates(subset=["primaryTitle"])

In [ ]:
# find the duplicate rows based on all columns
wiki[wiki["primaryTitle"] == "interstellar"]

In [ ]:
pd.merge(train, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

In [ ]:
#final_train = pd.merge(train, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

final_train = train.merge(wiki.select('primaryTitle', 'Genre', 'startYear'), on=['primaryTitle', 'startYear'], how='left')
final_train

In [ ]:
#final_train.to_csv("../datasets/gpt_data/train_plots_awards_genre_no_duplicates.csv", index=False)

final_train.toPandas().to_csv('../datasets/gpt_data/train_plots_awards_genre_no_duplicates.csv', index=False)

### Validation

In [1]:
# validation = pd.read_csv("../datasets/gpt_data/validation_plots_awards.csv", sep=',')

validation = spark.read.option("escape", "\"").csv("../datasets/gpt_data/validation_plots_awards.csv", sep=",", header=True, inferSchema=True)
validation

NameError: name 'spark' is not defined

In [ ]:
#validation["startYear"].replace(r"\N", None, inplace=True)
#validation["startYear"].fillna(validation['endYear'], inplace=True)
#validation["startYear"] = validation["startYear"].astype(int)

validation = validation.withColumn("startYear", regexp_replace(col("startYear"), r"\\N", str(None)))
validation = validation.fillna({'startYear': 'endYear'}, subset=['startYear'])
#validation = validation.select("startYear").fillna(validation.select("endYear"))
validation = validation.withColumn("startYear", col("startYear").cast("int"))

In [ ]:
#validation["primaryTitle"] = validation["primaryTitle"].str.lower()
#validation['primaryTitle'] = validation['primaryTitle'].apply(unidecode)

validation = validation.withColumn("primaryTitle", lower(col("primaryTitle")))
validation = validation.withColumn("primaryTitle", unidecode(validation["primaryTitle"]))

In [ ]:
pd.merge(validation, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

In [ ]:
#validation_final = pd.merge(validation, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

validation_final = validation.merge(wiki.select('primaryTitle', 'Genre', 'startYear', on=['primaryTitle', 'startYear'], how='left'))
validation_final

In [ ]:
#validation_final.to_csv("../datasets/gpt_data/validation_plots_awards_genre_no_duplicates.csv", index=False)

validation.toPandas().to_csv("../datasets/gpt_data/validation_plots_awards_genre_no_duplicates.csv", index=False)

### Test data

In [ ]:
# test = pd.read_csv("../datasets/gpt_data/test_plots_awards.csv")
test = spark.read.option("escape", "\"").csv("../datasets/gpt_data/test_plots_awards.csv",
                                             sep=",", header=True, inferSchema=True)

In [ ]:
#test["startYear"].replace(r"\N", None, inplace=True)
#test["startYear"].fillna(test['endYear'], inplace=True)
#test["startYear"] = test["startYear"].astype(int)
#test["primaryTitle"] = test["primaryTitle"].str.lower()
#test['primaryTitle'] = test['primaryTitle'].apply(unidecode)

test = test.withColumn("startYear", regexp_replace(col("startYear"), r"\\N", str(None)))
test = test.fillna({'startYear': 'endYear'}, subset=['startYear'])
#test = test.select("startYear").fillna(test.select("endYear"))
test = test.withColumn("startYear", col("startYear").cast("int"))
test = test.withColumn("primaryTitle", lower(col("primaryTitle")))
test = test.withColumn("primaryTitle", unidecode(test["primaryTitle"]))

In [ ]:
test

In [ ]:
pd.merge(test, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

In [ ]:
#test_final = pd.merge(test, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

test_final = test.merge(wiki.select('primaryTitle', 'Genre', 'startYear'), on=['primaryTitle', 'startYear'], how='left')
test_final

In [ ]:
#test_final.to_csv("../datasets/gpt_data/test_plots_awards_genre_no_duplicates.csv", index=False)

test_final.toPandas().to_csv('../datasets/gpt_data/test_plots_awards_genre_no_duplicates.csv', index=False)

In [9]:
pd.merge(test, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,plot,awards,Genre
0,22,tt0014972,he who gets slapped,He Who Gets Slapped,1924,\N,95,3654.0,He Who Gets Slapped is a 1924 silent film di...,0,drama
1,23,tt0015016,the iron horse,NaN,1924,\N,150,2136.0,The Iron Horse is a 1924 American silent wes...,0,western
2,26,tt0015174,die nibelungen: kriemhild's revenge,NaN,1924,\N,129,4341.0,The Die Nibelungen: Kriemhild's Revenge is a...,0,NaN
3,28,tt0015214,at 3:25,NaN,1925,1925,59,1724.0,At 3:25 is a drama film about a young woman ...,0,NaN
4,34,tt0015863,go west,NaN,1925,\N,69,4188.0,Go West is a 1925 silent comedy film starrin...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1081,9942,tt9430698,one piece: stampede,NaN,2019,\N,101,5109.0,The Straw Hat Pirates are invited to the Pir...,0,NaN
1082,9943,tt9441638,the big ugly,NaN,2020,\N,106,5780.0,The Big Ugly is a 2020 crime drama set in th...,0,NaN
1083,9948,tt9495690,pagalpanti,Pagalpanti,2019,\N,149,2331.0,The Págálpánti is a 2019 Indian drama film d...,0,NaN
1084,9950,tt9519642,the wedding unplanner,NaN,2020,\N,110,NaN,The Wedding Unplanner is a 2020 romantic com...,0,NaN


In [10]:
test_final = pd.merge(test, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")
test_final

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,plot,awards,Genre
0,22,tt0014972,he who gets slapped,He Who Gets Slapped,1924,\N,95,3654.0,He Who Gets Slapped is a 1924 silent film di...,0,drama
1,23,tt0015016,the iron horse,NaN,1924,\N,150,2136.0,The Iron Horse is a 1924 American silent wes...,0,western
2,26,tt0015174,die nibelungen: kriemhild's revenge,NaN,1924,\N,129,4341.0,The Die Nibelungen: Kriemhild's Revenge is a...,0,NaN
3,28,tt0015214,at 3:25,NaN,1925,1925,59,1724.0,At 3:25 is a drama film about a young woman ...,0,NaN
4,34,tt0015863,go west,NaN,1925,\N,69,4188.0,Go West is a 1925 silent comedy film starrin...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1081,9942,tt9430698,one piece: stampede,NaN,2019,\N,101,5109.0,The Straw Hat Pirates are invited to the Pir...,0,NaN
1082,9943,tt9441638,the big ugly,NaN,2020,\N,106,5780.0,The Big Ugly is a 2020 crime drama set in th...,0,NaN
1083,9948,tt9495690,pagalpanti,Pagalpanti,2019,\N,149,2331.0,The Págálpánti is a 2019 Indian drama film d...,0,NaN
1084,9950,tt9519642,the wedding unplanner,NaN,2020,\N,110,NaN,The Wedding Unplanner is a 2020 romantic com...,0,NaN


In [11]:
test_final.to_csv("../datasets/gpt_data/test_plots_awards_genre_no_duplicates.csv", index=False)